In [3]:
import numpy as np
def paa(x, w):
    #w is the number of segments
    #first calculate the length of the time series
    n = len(x)
    #now calculate the length of each segment
    step = n/w
    #now calculate the paa
    paa = []
    for i in range(w):
        paa.append(np.mean(x[int(i*step):int((i+1)*step)]))
    return paa


def envelopes(X, k):
    #first we recieve a time series X, and we compute the upper and lower envelope of X
    env_lower = []
    env_upper = []
    
    #calcalating lower envelope
    for i in range(0, len(X)):
        lowest = 1000
        for j in range(-k,k+1):
            if(i+j >= 0 and i+j < len(X)):
                if(X[i+j] < lowest):
                    lowest = X[i+j]
        env_lower.append(lowest)
    
    #calcalating upper envelope
    for i in range(0, len(X)):
        highest = 0
        for j in range(-k,k+1):
            if(i+j >= 0 and i+j < len(X)):
                if(X[i+j] > highest):
                    highest = X[i+j]
        env_upper.append(highest)
    
    return env_lower, env_upper

In [4]:
from scipy.spatial import KDTree
import numpy as np
import os

ucr_segments = []
for filename in os.listdir("./test_dataset/UCR"):
    f =open("./test_dataset/UCR/" + filename + "/" + filename+"_TEST")
    content = f.read()
    content = content.split("\n")
    content = content[0]
    content = content.split(",")
    content = content[1:]
    content = list(map(float, content))
    content = paa(content,4)
    ucr_segments.append(content)
    f.close()

ucr_tree = KDTree(ucr_segments,leafsize=2, balanced_tree=True)
print("UCR tree is built")


UCR tree is built


In [5]:
#generate a random time series


In [6]:
import pretty_midi

index = []
files = {}
counter = 0
for filename in os.listdir("./test_dataset/dataset_100/"):
    files[counter] = filename
    try:
        midi_data = pretty_midi.PrettyMIDI("test_dataset/dataset_100/" + filename)
    except:
        continue
    song_duration = midi_data.get_end_time()
    segment_duration = 15
    tempo = midi_data.estimate_tempo()
 
    
    for i in range(0, int(song_duration) - segment_duration):
        prev = 1
        segment = []
        for j in range(prev, len(midi_data.instruments[0].notes)):
            note = midi_data.instruments[0].notes[j]
            if note.start >= i and note.end <= i + segment_duration:
                segment.append(note.pitch - midi_data.instruments[0].notes[j - 1].pitch)
                #segment.append(note.pitch)
        prev = j + 1
        segment = paa(segment, 8)
        #if segment contiain nan, skip it
        if np.isnan(segment).any():
            continue
        counter += 1
        index.append(segment)
        # try:
        #     midi_idx.insert(counter, (segment_4[0], segment_4[1], segment_4[2], segment_4[3]),obj=filename)
        #     counter+=1
        # except:
        #     continue
    
    f.close()

midi_tree = KDTree(index, leafsize=2)
print("midi tree is built")
print(files)


c:\Users\soumi\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\soumi\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\Users\soumi\AppData\Local\Programs\Python\Python310\lib\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


midi tree is built
{0: 'ABBA_-_Chiquita.mid', 263: 'ABBA_-_Dancing_Queen.mid', 484: 'ABBA_-_Fernando.mid', 677: 'ABBA_-_Gimme_Gimme_Gimme.mid', 859: 'ABBA_-_Knowing_Me_Knowing_You.mid', 1078: 'ABBA_-_The_Winner_Takes_It_All.mid', 1352: 'ABBA_-_Waterloo.mid', 1498: 'Air_Supply_-_All_Out_Of_Love.mid', 1702: 'Air_Supply_-_Making_Love_Out_Of_Nothing.mid', 1990: 'Backstreet_Boys_-_As_Long_As_You_Love_Me.mid', 2160: 'Backstreet_Boys_-_I_Want_It_That_Way (1).mid', 2270: 'Backstreet_Boys_-_I_Want_It_That_Way.mid', 2380: 'Backstreet_Boys_-_Shape_of_My_Heart.mid', 2582: 'Billy_Joel_-_Piano_Man.mid', 2900: 'Billy_Joel_-_Uptown_Girl.mid', 3077: 'Boyzone_-_I_Love_The_Way_You_Love_Me.mid', 3284: 'Boyzone_-_Picture_Of_You.mid', 3416: 'Boyzone_-_Words.mid', 3641: 'Bruno_Mars_-_When_i_was_your_man.mid', 3830: 'Disney_Themes_-_Hakuna_Matata.mid', 3861: 'Disney_Themes_-_The_Sorcerer.mid', 3911: 'Eminem_-_My_Name_Is.mid', 4042: 'Eminem_-_Stan.mid', 4372: 'Eminem_-_Superman.mid', 4416: 'Eminem_-_The_Real_S

In [13]:
import feature_extraction.extract as extract
import input_query.record as record
#record_input = record.RecordQuery(15)
absolute_pitch, relative_pitch, tempo = extract.extract("user_input/query.wav", apitch=True, tempo=True, rpitch=True)
paa_4 = paa(relative_pitch, 8)
lower,upper = envelopes(relative_pitch, 10)
paa_lower = paa(lower, 8)
paa_upper = paa(upper, 8)

#give the query to the tree between paa_lower and paa_upper
dist,ind = midi_tree.query([paa_4],k=1000)


Predicting MIDI for user_input/query.wav...


In [14]:
#now we have this ind matrix, we need to extract the file names
#we can use the files dictionary

#create a empty set
song_list = {}
for i in ind[0]:
    #find i in files, if not found, find closest one before
    if i in files:
        if files[i] not in song_list:
            song_list[files[i]] = 1
        else:
            song_list[files[i]] += 1
    else:
        for j in range(i,0,-1):
            if j in files:
                if files[j] not in song_list:
                    song_list[files[j]] = 1
                else:
                    song_list[files[j]] += 1
                break

#sort the song dictionary by value descending
sorted_song_list = sorted(song_list.items(), key=lambda kv: kv[1], reverse=True)

#remove songs with less than 10 matches
sorted_song_list = [i for i in sorted_song_list if i[1] >= 10]
print(len(sorted_song_list))
print(sorted_song_list)


44
[('Billy_Joel_-_Piano_Man.mid', 37), ('ABBA_-_Knowing_Me_Knowing_You.mid', 35), ("Snoop_Dogg_-_What's_My_Name.mid", 34), ('The_Beatles_-_Drive_My_Car.mid', 30), ('The_Beatles_-_Long_and_Winding_Road.mid', 25), ('The_Beatles_-_Helter_Skelter.mid', 25), ('The_Beatles_-_Twist_and_Shout.mid', 25), ('Eminem_-_The_Real_Slim_Shady.mid', 24), ('The_Smiths_-_How_Soon_Is_Now.mid', 21), ('Boyzone_-_I_Love_The_Way_You_Love_Me.mid', 21), ('Wild_Cherry_-_Play_That_Funky_Music.mid', 20), ('one_direction_-_What_Makes_You_Beautiful.mid', 20), ('The_Beatles_-_The_Ballad_of_John_and_Yoko.mid', 19), ('ABBA_-_The_Winner_Takes_It_All.mid', 19), ('The_Beatles_-_A_Day_in_the_Life.mid', 18), ('ABBA_-_Gimme_Gimme_Gimme.mid', 17), ('ABBA_-_Dancing_Queen.mid', 17), ("The_Beatles_-_A_Hard_Day's_Night.mid", 17), ('Movie_Themes_-_Mortal_Kombat.mid', 16), ('Queen_-_Too_Much_Love_Will_Kill_You.mid', 16), ('The_Beatles_-_Got_to_Get_You_Into_My_Life.mid', 16), ('The_Beatles_-_Eight_Days_A_Week.mid', 16), ('The_Beatle

In [15]:
from fastdtw import fastdtw
song_list = [i[0] for i in sorted_song_list]

results = []
for i in song_list:
    midi_data = pretty_midi.PrettyMIDI("test_dataset/dataset_100/" + i)
    song_duration = midi_data.get_end_time()
    segment_duration = 15
    smallest=1000000
    for j in range(0, int(song_duration) - segment_duration):
        prev = 1
        segment = []
        for k in range(prev, len(midi_data.instruments[0].notes)):
            note = midi_data.instruments[0].notes[k]
            if note.start >= j and note.end <= j + segment_duration:
                segment.append(note.pitch - midi_data.instruments[0].notes[k - 1].pitch)
                #segment.append(note.pitch)
        prev = k + 1
        #if segment contiain nan, skip it
        if np.isnan(segment).any():
            continue
        if len(segment)<4:
            continue
        distance, path = fastdtw(segment,relative_pitch)
        if distance < smallest:
            smallest= distance
    results.append({"filename":i, "distance":smallest})
results = sorted(results, key=lambda k: k['distance'])
    

c:\Users\soumi\AppData\Local\Programs\Python\Python310\lib\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


In [16]:
for i in results:
    print(i)

{'filename': 'one_direction_-_History.mid', 'distance': 74.0}
{'filename': 'The_Beatles_-_We_Can_Work_It_Out.mid', 'distance': 74.0}
{'filename': 'The_Smiths_-_How_Soon_Is_Now.mid', 'distance': 77.0}
{'filename': 'Queen_-_Too_Much_Love_Will_Kill_You.mid', 'distance': 80.0}
{'filename': 'The_Beatles_-_The_Ballad_of_John_and_Yoko.mid', 'distance': 81.0}
{'filename': 'Sonny_and_Cher_-_All_I_Ever_Need_Is_You.mid', 'distance': 81.0}
{'filename': 'Queen_-_i_want_to_break_free.mid', 'distance': 81.0}
{'filename': 'The_Beatles_-_All_You_Need_Is_Love.mid', 'distance': 83.0}
{'filename': 'The_Beatles_-_Here_Comes_the_Sun.mid', 'distance': 83.0}
{'filename': 'Bruno_Mars_-_When_i_was_your_man.mid', 'distance': 84.0}
{'filename': 'The_Beatles_-_Helter_Skelter.mid', 'distance': 85.0}
{'filename': 'ABBA_-_Gimme_Gimme_Gimme.mid', 'distance': 85.0}
{'filename': 'The_Beatles_-_Get_Back.mid', 'distance': 85.0}
{'filename': 'The_Beatles_-_Penny_Lane.mid', 'distance': 85.0}
{'filename': 'The_Beatles_-_She_